In [1]:
!pip install transformers flair

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.1/373.1 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 kB 30.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 73.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 40.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting req

In [2]:
import pandas as pd
from transformers import pipeline
from flair.models import TextClassifier
classifier = TextClassifier.load('en-sentiment')
from flair.data import Sentence
from sklearn.metrics import accuracy_score

2023-05-21 18:31:06,991 https://nlp.informatik.hu-berlin.de/resources/models/sentiment-curated-distilbert/sentiment-en-mix-distillbert_4.pt not found in cache, downloading to /tmp/tmpos2lzctl


100%|██████████| 253M/253M [00:06<00:00, 38.5MB/s]

2023-05-21 18:31:13,970 copying /tmp/tmpos2lzctl to cache at /root/.flair/models/sentiment-en-mix-distillbert_4.pt


2023-05-21 18:31:14,578 removing temp file /tmp/tmpos2lzctl


In [3]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00331/sentiment%20labelled%20sentences.zip

--2023-05-21 18:31:34--  https://archive.ics.uci.edu/ml/machine-learning-databases/00331/sentiment%20labelled%20sentences.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84188 (82K) [application/x-httpd-php]
Saving to: ‘sentiment labelled sentences.zip’

sentiment labelled  100%[===================>]  82.21K   267KB/s    in 0.3s    

2023-05-21 18:31:35 (267 KB/s) - ‘sentiment labelled sentences.zip’ saved [84188/84188]



In [4]:
from zipfile import ZipFile
with ZipFile('sentiment labelled sentences.zip', 'r') as z:
  z.extractall()

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Proszę zmienić ścieżkę do dysku i wybrać odpowiedni plik

In [10]:
drive_path = "/content/drive/MyDrive/ZUM/projekt/"

file_name = "100-tweets.csv"    # 100 tweets
# file_name = "20k-tweets.csv"    # 23392 tweets
# file_name = "200k-tweets.csv"   # 231423 tweets

tweets_path = f"{drive_path}{file_name}"

In [11]:
tweets_df = pd.read_csv(tweets_path)

tweets_df.head()

,id,text,user_id,user_name
0,1656548085038989312,RT @DianneCostin: Why is Australia even in the...,517101711,yabbietol
1,1656548073399525376,RT @MonaLisa_1797: IT'S CRAZY IT'S PARTY IT'S ...,4813234648,sergilunary
2,1656548064927023110,RT @_aleexxtr: Mae Muller - I Wrote a Song | U...,725805985,winnyesc
3,1656548052109254657,@jillyluvsmusic Happy birthday Jilly . Have a ...,291837741,angelathompson5
4,1656548046119870465,happy semi final 2 of #eurovision day to those...,1333144158828064768,eurovanya


In [12]:
classifier = pipeline(task="zero-shot-classification", model="facebook/bart-large-mnli", device=0) 

Tagowanie

In [13]:
sequences = tweets_df['text'].to_list()

candidate_labels = ["positive", "negative"]
hypothesis_template = "The sentiment of this tweet is {}."

hf_prediction = classifier(sequences, candidate_labels, hypothesis_template=hypothesis_template)
hf_prediction = pd.DataFrame(hf_prediction)

hf_prediction.head()

,sequence,labels,scores
0,RT @DianneCostin: Why is Australia even in the...,"[negative, positive]","[0.9889222383499146, 0.01107778586447239]"
1,RT @MonaLisa_1797: IT'S CRAZY IT'S PARTY IT'S ...,"[negative, positive]","[0.7381644248962402, 0.26183560490608215]"
2,RT @_aleexxtr: Mae Muller - I Wrote a Song | U...,"[positive, negative]","[0.8818310499191284, 0.11816894263029099]"
3,@jillyluvsmusic Happy birthday Jilly . Have a ...,"[positive, negative]","[0.9966152906417847, 0.003384695854038]"
4,happy semi final 2 of #eurovision day to those...,"[positive, negative]","[0.9983842372894287, 0.0016157389618456364]"


In [14]:
hf_prediction['hf_prediction'] = hf_prediction['labels'].apply(lambda x: x[0])
hf_prediction['hf_prediction'] = hf_prediction['hf_prediction'].map({'positive': 1, 'negative': 0})
hf_prediction['hf_predicted_score'] = hf_prediction['scores'].apply(lambda x: x[0])

hf_prediction.head()

,sequence,labels,scores,hf_prediction,hf_predicted_score
0,RT @DianneCostin: Why is Australia even in the...,"[negative, positive]","[0.9889222383499146, 0.01107778586447239]",0,0.988922
1,RT @MonaLisa_1797: IT'S CRAZY IT'S PARTY IT'S ...,"[negative, positive]","[0.7381644248962402, 0.26183560490608215]",0,0.738164
2,RT @_aleexxtr: Mae Muller - I Wrote a Song | U...,"[positive, negative]","[0.8818310499191284, 0.11816894263029099]",1,0.881831
3,@jillyluvsmusic Happy birthday Jilly . Have a ...,"[positive, negative]","[0.9966152906417847, 0.003384695854038]",1,0.996615
4,happy semi final 2 of #eurovision day to those...,"[positive, negative]","[0.9983842372894287, 0.0016157389618456364]",1,0.998384


In [15]:
hf_prediction['hf_prediction']

0     0
1     0
2     1
3     1
4     1
     ..
95    1
96    0
97    1
98    0
99    0
Name: hf_prediction, Length: 100, dtype: int64

Przypisanie tagowania

In [16]:
tweets_df['label'] = hf_prediction['hf_prediction'].values
tweets_df['label_score'] = hf_prediction['hf_predicted_score'].values

tweets_df

,id,text,user_id,user_name,label,label_score
0,1656548085038989312,RT @DianneCostin: Why is Australia even in the...,517101711,yabbietol,0,0.988922
1,1656548073399525376,RT @MonaLisa_1797: IT'S CRAZY IT'S PARTY IT'S ...,4813234648,sergilunary,0,0.738164
2,1656548064927023110,RT @_aleexxtr: Mae Muller - I Wrote a Song | U...,725805985,winnyesc,1,0.881831
3,1656548052109254657,@jillyluvsmusic Happy birthday Jilly . Have a ...,291837741,angelathompson5,1,0.996615
4,1656548046119870465,happy semi final 2 of #eurovision day to those...,1333144158828064768,eurovanya,1,0.998384
...,...,...,...,...,...,...
95,1656546983706755073,RT @colinwright1959: Last night’s show and all...,1563812233641091072,CBWRadioUK,1,0.992593
96,1656546969341272065,I was reading a sad report about how a bloke f...,1176815134817345537,GeezerDel,0,0.985566
97,1656546955210702851,RT @SweetCherryPub: The Songs You've Never Hea...,1181492890356387840,JulieEd61627334,1,0.714881
98,1656548256875257857,RT @VFinnishProbs: You know the #Eurovision t...,1121178672218759168,joeykux,0,0.903400


Czyszczenie danych

Do małych liter

In [17]:
tweets_df['text'] = tweets_df['text'].apply(lambda x: x.lower())
tweets_df.head()

,id,text,user_id,user_name,label,label_score
0,1656548085038989312,rt @diannecostin: why is australia even in the...,517101711,yabbietol,0,0.988922
1,1656548073399525376,rt @monalisa_1797: it's crazy it's party it's ...,4813234648,sergilunary,0,0.738164
2,1656548064927023110,rt @_aleexxtr: mae muller - i wrote a song | u...,725805985,winnyesc,1,0.881831
3,1656548052109254657,@jillyluvsmusic happy birthday jilly . have a ...,291837741,angelathompson5,1,0.996615
4,1656548046119870465,happy semi final 2 of #eurovision day to those...,1333144158828064768,eurovanya,1,0.998384


Usunięcie oznaczeń i hiperłączy

In [18]:
import re

In [19]:
re_url = re.compile(r'((@[^\s]+)|(#[^\s]+)|(www.[^\s+]+)|(https?://[^\s]+))')
tweets_df['text'] = tweets_df['text'].apply(lambda x: re_url.sub('', x))
tweets_df.head()

,id,text,user_id,user_name,label,label_score
0,1656548085038989312,rt why is australia even in the eurovision so...,517101711,yabbietol,0,0.988922
1,1656548073399525376,rt it's crazy it's party it's cha cha cha!!! ...,4813234648,sergilunary,0,0.738164
2,1656548064927023110,rt mae muller - i wrote a song | united kingd...,725805985,winnyesc,1,0.881831
3,1656548052109254657,happy birthday jilly . have a great day in li...,291837741,angelathompson5,1,0.996615
4,1656548046119870465,happy semi final 2 of day to those who celebr...,1333144158828064768,eurovanya,1,0.998384


Pozostawienie samych liter

In [20]:
re_character = re.compile(r"[^a-zA-Z\s']")
tweets_df['text'] = tweets_df['text'].apply(lambda x: re_character.sub('', x))
tweets_df.head()

,id,text,user_id,user_name,label,label_score
0,1656548085038989312,rt why is australia even in the eurovision so...,517101711,yabbietol,0,0.988922
1,1656548073399525376,rt it's crazy it's party it's cha cha cha,4813234648,sergilunary,0,0.738164
2,1656548064927023110,rt mae muller i wrote a song united kingdom...,725805985,winnyesc,1,0.881831
3,1656548052109254657,happy birthday jilly have a great day in liv...,291837741,angelathompson5,1,0.996615
4,1656548046119870465,happy semi final of day to those who celebrate,1333144158828064768,eurovanya,1,0.998384


In [21]:
# POTRZEBOWAŁEM DO POBRANIA STOPWORDÓW
# PANI MOŻE NIE MUSI TEGO ODPALAĆ

import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [22]:
!pip install -U spacy
!python -m spacy download en_core_web_md

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 63.1 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.5.2
    Uninstalling spacy-3.5.2:
      Successfully uninstalled spacy-3.5.2
2023-05-21 18:53:18.447861: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 16.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


Ściągnięcie stopwordów

In [23]:
import spacy
nlp = spacy.load("en_core_web_md", disable=['ner', 'parser'])
nlp.add_pipe('sentencizer')

In [24]:
stop_words = nlp.Defaults.stop_words.copy()
stop_words.add('rt')

# dodałem "rt" do zbioru stopwordów ("rt" oznacza "retweet")

Usunięcie stopwordów

In [25]:
def remove_stopwords(text):
  return ' '.join(filter(lambda x: x not in stop_words, text.split()))

tweets_df['text'] = tweets_df['text'].apply(remove_stopwords)
tweets_df.head()

,id,text,user_id,user_name,label,label_score
0,1656548085038989312,australia eurovision song contest,517101711,yabbietol,0,0.988922
1,1656548073399525376,it's crazy it's party it's cha cha cha,4813234648,sergilunary,0,0.738164
2,1656548064927023110,mae muller wrote song united kingdom dress reh...,725805985,winnyesc,1,0.881831
3,1656548052109254657,happy birthday jilly great day liverpool eurov...,291837741,angelathompson5,1,0.996615
4,1656548046119870465,happy semi final day celebrate,1333144158828064768,eurovanya,1,0.998384


Lematyzacja

In [26]:
def lemmatize(text):
  return ' '.join([x.lemma_ for x in nlp(text)])

tweets_df['text'] = tweets_df['text'].apply(lemmatize)
tweets_df.head()

,id,text,user_id,user_name,label,label_score
0,1656548085038989312,australia eurovision song contest,517101711,yabbietol,0,0.988922
1,1656548073399525376,it be crazy it be party it be cha cha cha,4813234648,sergilunary,0,0.738164
2,1656548064927023110,mae muller write song united kingdom dress reh...,725805985,winnyesc,1,0.881831
3,1656548052109254657,happy birthday jilly great day liverpool eurov...,291837741,angelathompson5,1,0.996615
4,1656548046119870465,happy semi final day celebrate,1333144158828064768,eurovanya,1,0.998384


In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [28]:
X_train, X_test, y_train, y_test = train_test_split(tweets_df.text, tweets_df.label, test_size=0.1, stratify=tweets_df.label, random_state=42)

In [29]:
vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features=500000)
vectorizer.fit(X_train)

TfidfVectorizer(max_features=500000, ngram_range=(1, 2))

In [30]:
X_train = vectorizer.transform(X_train)
X_test = vectorizer.transform(X_test)

In [31]:
from sklearn.naive_bayes import BernoulliNB, GaussianNB, MultinomialNB
from sklearn.metrics import confusion_matrix, classification_report, roc_curve

Bernoulli

In [32]:
model_bernoulli = BernoulliNB()
model_bernoulli.fit(X_train, y_train)

BernoulliNB()

In [36]:
y_pred_bernoulli = model_bernoulli.predict(X_test)
print(classification_report(y_test, y_pred_bernoulli))

              precision    recall  f1-score   support

           0       0.62      1.00      0.77         5
           1       1.00      0.40      0.57         5

    accuracy                           0.70        10
   macro avg       0.81      0.70      0.67        10
weighted avg       0.81      0.70      0.67        10



In [43]:
confusion_matrix(y_test, y_pred_bernoulli)

array([[5, 0],
       [3, 2]])

In [38]:
roc_curve(y_test, y_pred_bernoulli)

(array([0., 0., 1.]), array([0. , 0.4, 1. ]), array([2, 1, 0]))

Gaussian

In [40]:
model_gaussian = GaussianNB()
model_gaussian.fit(X_train.toarray(), y_train)

GaussianNB()

In [42]:
y_pred_gaussian = model_gaussian.predict(X_test.toarray())
print(classification_report(y_test, y_pred_gaussian))

              precision    recall  f1-score   support

           0       0.25      0.20      0.22         5
           1       0.33      0.40      0.36         5

    accuracy                           0.30        10
   macro avg       0.29      0.30      0.29        10
weighted avg       0.29      0.30      0.29        10



In [48]:
confusion_matrix(y_test, y_pred_gaussian)

array([[1, 4],
       [3, 2]])

In [50]:
roc_curve(y_test, y_pred_gaussian)

(array([0. , 0.8, 1. ]), array([0. , 0.4, 1. ]), array([2, 1, 0]))

Multinomial

In [46]:
model_multinomial = MultinomialNB()
model_multinomial.fit(X_train, y_train)

MultinomialNB()

In [47]:
y_pred_multinomial = model_multinomial.predict(X_test)
print(classification_report(y_test, y_pred_multinomial))

              precision    recall  f1-score   support

           0       0.57      0.80      0.67         5
           1       0.67      0.40      0.50         5

    accuracy                           0.60        10
   macro avg       0.62      0.60      0.58        10
weighted avg       0.62      0.60      0.58        10



In [49]:
confusion_matrix(y_test, y_pred_multinomial)

array([[4, 1],
       [3, 2]])

In [51]:
roc_curve(y_test, y_pred_multinomial)

(array([0. , 0.2, 1. ]), array([0. , 0.4, 1. ]), array([2, 1, 0]))

BLSMT

In [52]:
import keras
from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences, to_categorical

In [53]:
max_words = 5000
max_len = 200

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(tweets_df.text)
sequences = tokenizer.texts_to_sequences(tweets_df.text)
tweets = pad_sequences(sequences, maxlen=max_len)

In [54]:
labels = to_categorical(tweets_df.label, num_classes=2)

In [55]:
X_train, X_test, y_train, y_test = train_test_split(tweets, labels, test_size=0.1, stratify=labels, random_state=42)

In [57]:
from keras.callbacks import EarlyStopping
es = EarlyStopping(patience=5, monitor='val_loss')

In [59]:
model_blsmt = Sequential()
model_blsmt.add(layers.Embedding(max_words, 40))
model_blsmt.add(layers.Bidirectional(layers.LSTM(20, dropout=0.5)))
model_blsmt.add(layers.Dense(2, activation="softmax"))

model_blsmt.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])

history_blsmt = model_blsmt.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), callbacks=[es])

Epoch 1/100
3/3 [==============================] - 9s 611ms/step - loss: 0.6932 - accuracy: 0.4889 - val_loss: 0.6916 - val_accuracy: 0.5000
Epoch 2/100
3/3 [==============================] - 1s 201ms/step - loss: 0.6854 - accuracy: 0.6111 - val_loss: 0.6915 - val_accuracy: 0.5000
Epoch 3/100
3/3 [==============================] - 0s 100ms/step - loss: 0.6826 - accuracy: 0.5556 - val_loss: 0.6910 - val_accuracy: 0.5000
Epoch 4/100
3/3 [==============================] - 0s 123ms/step - loss: 0.6759 - accuracy: 0.5556 - val_loss: 0.6897 - val_accuracy: 0.5000
Epoch 5/100
3/3 [==============================] - 0s 204ms/step - loss: 0.6714 - accuracy: 0.5556 - val_loss: 0.6888 - val_accuracy: 0.5000
Epoch 6/100
3/3 [==============================] - 0s 155ms/step - loss: 0.6644 - accuracy: 0.5556 - val_loss: 0.6871 - val_accuracy: 0.5000
Epoch 7/100
3/3 [==============================] - 0s 51ms/step - loss: 0.6560 - accuracy: 0.5667 - val_loss: 0.6848 - val_accuracy: 0.5000
Epoch 8/100
3/

Sentiment analysis classifier

In [60]:
from datasets import Dataset
tweets_dataset_obj = Dataset.from_pandas(tweets_df)
tweets_dataset_obj

Dataset({
    features: ['id', 'text', 'user_id', 'user_name', 'label', 'label_score'],
    num_rows: 100
})

In [61]:
labels = to_categorical(tweets_df.label, num_classes=2)
tweets_dataset = tweets_dataset_obj.train_test_split(0.1)

In [62]:
tweets_dataset['train'][0]

{'id': 1656547958085525510,
 'text': 'sorry dailymail eurovision antibrexit it be continent come despite',
 'user_id': 861485320302415872,
 'user_name': 'HecSkill',
 'label': 0,
 'label_score': 0.9873778223991394}

In [63]:
model_checkpoint = "distilbert-base-uncased"
batch_size = 128

In [64]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [65]:
def process(x):
  return tokenizer(x['text'])

In [66]:
train_tds = tweets_dataset['train'].map(process)
test_tds = tweets_dataset['test'].map(process)

Map:   0%|          | 0/90 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [67]:
test_tds

Dataset({
    features: ['id', 'text', 'user_id', 'user_name', 'label', 'label_score', 'input_ids', 'attention_mask'],
    num_rows: 10
})

In [68]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)

In [69]:
args = TrainingArguments(
    "model_checkpoint", 
    evaluation_strategy='epoch', 
    save_strategy='epoch', 
    learning_rate=2e-5, 
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

In [70]:
from datasets import load_metric
metric = load_metric('glue', 'sst2')

<ipython-input-70-a646943e8c66>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('glue', 'sst2')


In [71]:
import numpy as np

def compute_metrics(eval_preds):
  logits, labels = eval_preds
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)

In [72]:
trainer = Trainer(model, args, train_dataset=train_tds, eval_dataset=test_tds, tokenizer=tokenizer, compute_metrics=compute_metrics)

In [73]:
res = trainer.evaluate([train_tds[0]])
res

{'eval_loss': 0.666271984577179, 'eval_accuracy': 1.0, 'eval_runtime': 0.0836, 'eval_samples_per_second': 11.955, 'eval_steps_per_second': 11.955}


{'eval_loss': 0.666271984577179,
 'eval_accuracy': 1.0,
 'eval_runtime': 0.0836,
 'eval_samples_per_second': 11.955,
 'eval_steps_per_second': 11.955}

In [74]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


{'eval_loss': 0.6735564470291138, 'eval_accuracy': 0.7, 'eval_runtime': 0.0489, 'eval_samples_per_second': 204.38, 'eval_steps_per_second': 20.438, 'epoch': 1.0}
{'eval_loss': 0.6592143774032593, 'eval_accuracy': 0.8, 'eval_runtime': 0.0271, 'eval_samples_per_second': 369.441, 'eval_steps_per_second': 36.944, 'epoch': 2.0}
{'eval_loss': 0.6502646803855896, 'eval_accuracy': 0.8, 'eval_runtime': 0.0264, 'eval_samples_per_second': 378.578, 'eval_steps_per_second': 37.858, 'epoch': 3.0}
{'eval_loss': 0.6444426774978638, 'eval_accuracy': 0.8, 'eval_runtime': 0.0281, 'eval_samples_per_second': 355.537, 'eval_steps_per_second': 35.554, 'epoch': 4.0}
{'eval_loss': 0.6422134041786194, 'eval_accuracy': 0.8, 'eval_runtime': 0.023, 'eval_samples_per_second': 433.937, 'eval_steps_per_second': 43.394, 'epoch': 5.0}
{'train_runtime': 28.5825, 'train_samples_per_second': 15.744, 'train_steps_per_second': 0.175, 'train_loss': 0.6810254573822021, 'epoch': 5.0}


TrainOutput(global_step=5, training_loss=0.6810254573822021, metrics={'train_runtime': 28.5825, 'train_samples_per_second': 15.744, 'train_steps_per_second': 0.175, 'train_loss': 0.6810254573822021, 'epoch': 5.0})

In [75]:
import torch

In [76]:
tokens = tokenizer([
    'Italy is not progressing at Eurovision', 
    'i love this excellent performance great', 
    'nobody want to listen this shit', 
    'great great great love from Mozambique'
])
tokens['input_ids']

[[101, 3304, 2003, 2025, 27673, 2012, 12714, 102],
 [101, 1045, 2293, 2023, 6581, 2836, 2307, 102],
 [101, 6343, 2215, 2000, 4952, 2023, 4485, 102],
 [101, 2307, 2307, 2307, 2293, 2013, 16274, 102]]

In [77]:
inputs = torch.tensor(tokens['input_ids']).cuda()

In [78]:
inputs

tensor([[  101,  3304,  2003,  2025, 27673,  2012, 12714,   102],
        [  101,  1045,  2293,  2023,  6581,  2836,  2307,   102],
        [  101,  6343,  2215,  2000,  4952,  2023,  4485,   102],
        [  101,  2307,  2307,  2307,  2293,  2013, 16274,   102]],
       device='cuda:0')

In [79]:
output=model(inputs)
output.logits

tensor([[ 0.0621,  0.0341],
        [-0.0741, -0.0630],
        [-0.0160, -0.0970],
        [-0.0263, -0.0166]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [80]:
output.logits.argmax(1)

tensor([0, 1, 0, 1], device='cuda:0')